In [1]:
import numpy as np
import nibabel as nib
from pathlib import Path

In [40]:
real_27 = nib.load(Path("..") / "data" / "segthor_train" / "train" / "Patient_27" / "GT2.nii.gz")
fake_27 = nib.load(Path("..") / "data" / "segthor_train" / "train" / "Patient_27" / "GT.nii.gz")
real_27_array = np.array(real_27.dataobj)
fake_27_array = np.array(fake_27.dataobj)

# look only to transform the heart
real_27_array = (real_27_array == 2)
fake_27_array = (fake_27_array == 2)

In [42]:
# find translation vector
def find_centroid(array):
    indices = np.argwhere(array == 1)  # Find points where array == 1
    centroid = np.mean(indices, axis=0)  # Compute mean of these points
    return centroid

centroid_real = find_centroid(real_27_array)
centroid_fake = find_centroid(fake_27_array)

translation_vector = centroid_real - centroid_fake

In [67]:
def test_rotation(real_array, fake_array, translation_vector, alpha, beta, gamma):
    rotation_matrix = np.asarray([[(np.cos(alpha) * np.cos(beta)) , np.sin(alpha)*np.sin(beta)*np.cos(gamma) - np.cos(alpha)*np.sin(gamma) , np.cos(alpha)*np.sin(beta)*np.cos(gamma) + np.sin(alpha)*np.sin(gamma) , 0],
                                 [(np.sin(alpha) * np.cos(beta))  , np.sin(alpha)*np.sin(beta)*np.sin(gamma) + np.cos(alpha)*np.cos(gamma) , np.cos(alpha)*np.sin(beta)*np.sin(gamma) + np.sin(alpha)*np.cos(gamma) , 0], 
                                 [     -np.sin(alpha)             , np.cos(alpha)*np.cos(beta)                                             , np.cos(alpha)*np.cos(beta)                                             , 0], 
                                 [     0,         0,     0, 1]])

    found_fake_array = affine_transform(
        fake_array, 
        rotation_matrix,
        offset=translation_vector,
        order=0 # uses nearest-neighbor interpolation
    )

    overlap = np.sum(real_array & found_fake_array)
                    
    return overlap, rotation_matrix

In [64]:
best_overlap = 0

# this would take a while
for alpha in range(0,360,0.1):
    for beta in range(0,360,0.1):
        for gamma in range(0,360,0.1):
            found_overlap, found_matrix = test_rotation(real_27_array, fake_27_array, translation_vector, alpha, beta, gamma)

            if found_overlap > best_overlap:
                rotation_matrix = found_matrix

In [5]:
def save_array_ass_nii(array, filename, fake_GT):
    """
    Converts a 3D NumPy array into a NIfTI (.nii) file.
    
    Args:
        array (np.ndarray): 3D NumPy array to be converted.
        filename (str): Path to save the NIfTI file (with .nii extension).
    
    Returns:
        None
    """
    # Convert NumPy array to a NIfTI image
    nii_image = nib.Nifti1Image(array, affine=fake_GT.affine, header=fake_GT.header)
    
    # Save the NIfTI image to a file
    nib.save(nii_image, filename)

In [16]:
# iterate over patients
for patient_number in range(1, 40):
    patient_number = "{:02d}".format(patient_number)
    print(f"working on patient {patient_number}")
    fake_nii = nib.load(Path("..") / "data" / "segthor_train" / "train" / f"Patient_{patient_number}" / "GT.nii.gz")
    fake_array = np.array(fake_nii.dataobj)

    # save non-heart organs
    saved_fake_array = np.copy(fake_array)
    saved_fake_array[saved_fake_array == 2] = 0

    # mask with only the heart
    fake_array = (fake_array == 2)

    found_fake_array = affine_transform(
        fake_array, 
        rotation_matrix,
        offset=translation_vector,
        order=0 # uses nearest-neighbor interpolation
    )
    
    # re-add the other organs
    saved_fake_array[found_fake_array == 1] = 2

    # save the found array
    save_array_ass_nii(saved_fake_array, Path("..") / "data" / "segthor_train" / "train" / f"Patient_{patient_number}" / "real_GT.nii.gz", fake_nii)
print("Done")

working on patient 01
working on patient 02
working on patient 03
working on patient 04
working on patient 05
working on patient 06
working on patient 07
working on patient 08
working on patient 09
working on patient 10
working on patient 11
working on patient 12
working on patient 13
working on patient 14
working on patient 15
working on patient 16
working on patient 17
working on patient 18
working on patient 19
working on patient 20
working on patient 21
working on patient 22
working on patient 23
working on patient 24
working on patient 25
working on patient 26
working on patient 27
working on patient 28
working on patient 29
working on patient 30
working on patient 31
working on patient 32
working on patient 33
working on patient 34
working on patient 35
working on patient 36
working on patient 37
working on patient 38
working on patient 39
Done
